In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import pickle
from gensim import corpora, models, similarities
def __pickleStuff(filename, stuff):
    save_stuff = open(filename, "wb")
    pickle.dump(stuff, save_stuff)
    save_stuff.close()
def __loadStuff(filename):
    saved_stuff = open(filename,"rb")
    stuff = pickle.load(saved_stuff)
    saved_stuff.close()
    return stuff

C:\Users\amrit\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
users = pd.read_csv("dbs.csv", error_bad_lines=False, delimiter=";")

In [3]:
users = users.drop(["youtubelink", "twitterlink", "facebooklink", "flickrlink"], axis=1)

In [4]:
users = users.dropna()

In [5]:
users_text = []

In [6]:
for index, row in users.iterrows():
    users_text.append(str(row["tagline"]) +". "+ str(row["profile"]))

In [67]:
test_output = __loadStuff("test_output.p")
test_output_more = __loadStuff("test_output_more.p")

In [83]:
" ".join(test_output.document[30])

"Google launched a new US wireless service today that switches between Wi-Fi network and cellular networks Google is already the world ’s most popular phone software provider , and a pay - tv operator - and now it wants to be your mobile network carrier the company has unveiled a US wireless service that switches between Wi-Fi network and cellular networks to curb data use and keep phone bills low the service , called ' Project Fi , ' debuted today , about two months after Google revealed its plans to expand its ever - growing empire into providing wireless connections for smartphones Google is selling the basic phone service for $ 20 a month and will only charge customers for the amount of cellular data that they use each month , instead of a flat rate each gigabyte of data will cost $ 10 a month that means a customer could sign up for a plan offering three gigabytes of data and get $ 20 back if only one gigabyte was used in a month most wireless phone carriers allow their customers t

In [84]:
test_output.summary[30]

"Project Fi will be hosted through Sprint Corp and T-Mobile 's networks it costs $ 20 for basic service and unused data is paid back to customer the invitation - only service will work only on Nexus 6 phones in the US numbers will live in the cloud so users can talk on any connected tablet"

In [85]:
test_output.predicted_summary[30]

"phone numbers will live in the cloud so that consumers can talk and text on any connected tablet. calls can be made via Hangouts on Android , iOS , and through Gmail on desktops , via the Hangouts widget. there is no annual service contract required when you sign up to use the service , Nexus 6 6 owners must sign - up to request an invitation , must have a Gmail address , and must live in a US zip code within the coverage area. however the savings really start when 5GB of LTE is consider - with the internet search giant 's plan coming in at $ 70 and telecom giant Verizon 's costing $ 110 a month Project Fi initially will only be sold to a narrow US audience that owns the Nexus 6 6 , a smartphone that Motorola Mobility made with Google 's help"

In [86]:
test_output_more.predicted_summary[30]

"Google launched a new US wireless service today that switches between Wi-Fi network and cellular networks Google is already the world ’s most popular phone software provider , and a pay - tv operator - and now it wants to be your mobile network carrier. the company has unveiled a US wireless service that switches between Wi-Fi network and cellular networks to curb data use and keep phone bills low. the service , called ' Project Fi , ' debuted today , about two months after Google revealed its plans to expand its ever - growing empire into providing wireless connections for smartphones. Google is selling the basic phone service for $ 20 a month and will only charge customers for the amount of cellular data that they use each month , instead of a flat rate. each gigabyte of data will cost $ 10 a month. Project Wi-Fi initially will only be sold to a narrow US audience that owns the Nexus 6 6 , a smartphone that Motorola Mobility made with Google 's help. Google 's pricing setup makes Pr

In [42]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [43]:
def preprocess(text, stopwords_removal=False, stemming=False):
    cachedStopWords = stopwords.words("english")
    text = text.lower()
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r" ' ", " ", text)
    str1 = re.findall("[@a-zA-Z0-9]* '[a-zA-Z]", text)
    for l in str1:
        text= re.sub(l,l.replace(" ",""),text)
    text = re.findall(r"[\w']+", text)
    new_text = []
    for word in text:
        if word in contractions:
            new_text.append(contractions[word])
        else:
            new_text.append(word)
    text = " ".join(new_text)
    tokens = text.split()
    words = []
    if stopwords_removal:
        for token in tokens:
            if token not in cachedStopWords:
                words.append(token)
            text = " ".join(words)        
    if stemming:
        text = [" ".join([stem(word) for word in sentence.split(" ")]) for sentence in text]
    return text

In [25]:
sentences = []
for i in range(len(test_output["document"])):
    for j in range(len(test_output["document"][i])):
        sentences.append(preprocess(test_output["document"][i][j], True))

In [44]:
users_text_pre = []
for i in range(len(users_text)):
    soup = BeautifulSoup(users_text[i])
    letters_only = re.sub("[^a-zA-Z]", " ", soup.get_text())
    users_text_pre.append(letters_only)

C:\Users\amrit\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\amrit\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
C:\Users\amrit\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.... ...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\amrit\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.. .'" looks like a filename, not markup. You should probably op

In [45]:
users_text[20832]

'Student (T.T.V.). Hey!! I am <b>Rajorshi Sanyal</b>. Nickname<b> Rupam</b>. I am a student. Fond of Playing Cricket, Football, Table Tennis<div>and Computer games.</div><div>Favourite Player: <b>MS- Dhoni</b> &amp; <b>Sourav Ganguly.</b></div><div>Favourite Footballer:<b> Cristiano Ronaldo</b> &amp; <b>Lionel Messi.</b></div><div><b><br></b></div><div><b><i><u><br></u></i></b></div>'

In [13]:
users_text_pre[12942]

'The Grumpy Old Troll  I comment on blogs and play PbP games as  migellito   and I write at The Grumpy Old Troll I also run an OD D game over G  Hangout every Monday night at  pm EST  Tuesday  am GMT  Tuesday  pm NZDT  Message me to get added to the player pool '

In [14]:
for i in range(len(users_text_pre)):
    if "ronaldo" in users_text_pre[i].split(" "):
        print(i)

In [46]:
users_text_pre[20832]

'Student  T T V    Hey   I am Rajorshi Sanyal  Nickname Rupam  I am a student  Fond of Playing Cricket  Football  Table Tennisand Computer games Favourite Player  MS  Dhoni   Sourav Ganguly Favourite Footballer  Cristiano Ronaldo   Lionel Messi '

In [47]:
users_text_pre[4333]

'i live  work and play in the digital world   i am a digital specialist in WNY with a focus on social media  content generation  inbound marketing  digital advertising and photography '

In [48]:
users_text_pre[4447]

'I live in the yonge and Eglinton area and I am laid back and easy to get a long with   I am here just seeing what this Google  thing is all about and maybe meet some cool people  I love playing guitar and hanging out with friends  I live in the Yonge and Eglinton area  I m friendly and easy to talk to    I am here to see what this Google  thing is all about and maybe meet some cool people  '

In [49]:
users_text_pre[5555]

'I am your atypical car and technology nut   Recovering Twitter Addict  I enjoy autos  travel  tech  and photography  I also enjoy reading fiction and learning new technology  I am a Microsoft Apple Google fan  Did I say I like cars  Estudiante de Espa ol  Soy latino honarario  '

In [50]:
users_text_pre[8888]

'iPHOTOSTUFF    T bone     Part time Aspirin Engraver    Seinfeld Fanatic    Jacked Up On Dew    Lover of all things shiny    No word wrap on this thing  G    D  Stay   Home Dad   It s all about Levels Jerry   Photographer of all things macro  get your TiMacrofied on on the www  http   timsdigitaldarkroom com in the blog matrix  http   www timsdd com defaced book  https   www facebook com pages Tims Digital Darkroom                '

In [51]:
users_text_pre[2323]

'Multimedia Developer  I have too many interests '

In [52]:
users_text_pre[9997]

'I can explain it to you but I can t understand if for you   Pastafarian  anatidaephobic and fucking cynic '

In [53]:
users_text_pre[4483]

'math  writing  tech  edu       Enjoy tinkering with mathematics  especially coding theory and representation theory   SAGE  www sagemath org  and GAP  www gap system org   writing  computers  video  chess  and photography  Interested in open source and math education related issues '

In [54]:
interesting_users = []

In [55]:
interesting_users.append(users_text_pre[2323])
interesting_users.append(users_text_pre[8888])
interesting_users.append(users_text_pre[5555])
interesting_users.append(users_text_pre[4483])
interesting_users.append(users_text_pre[4447])
interesting_users.append(users_text_pre[4333])

In [56]:
interesting_users

['Multimedia Developer  I have too many interests ',
 'iPHOTOSTUFF    T bone     Part time Aspirin Engraver    Seinfeld Fanatic    Jacked Up On Dew    Lover of all things shiny    No word wrap on this thing  G    D  Stay   Home Dad   It s all about Levels Jerry   Photographer of all things macro  get your TiMacrofied on on the www  http   timsdigitaldarkroom com in the blog matrix  http   www timsdd com defaced book  https   www facebook com pages Tims Digital Darkroom                ',
 'I am your atypical car and technology nut   Recovering Twitter Addict  I enjoy autos  travel  tech  and photography  I also enjoy reading fiction and learning new technology  I am a Microsoft Apple Google fan  Did I say I like cars  Estudiante de Espa ol  Soy latino honarario  ',
 'math  writing  tech  edu       Enjoy tinkering with mathematics  especially coding theory and representation theory   SAGE  www sagemath org  and GAP  www gap system org   writing  computers  video  chess  and photography  

In [57]:
from googleapiclient import discovery
import httplib2
from oauth2client.client import GoogleCredentials


# words = lower_case.split()
# words = [w for w in words if not w in stopwords.words("english")]
# print(lower_case)


def get_service():
    credentials = GoogleCredentials.get_application_default()
    scoped_credentials = credentials.create_scoped(
        ['https://www.googleapis.com/auth/cloud-platform'])
    http = httplib2.Http()
    scoped_credentials.authorize(http)
    return discovery.build('language', 'v1beta1', http=http)


def get_native_encoding_type():
    """Returns the encoding type that matches Python's native strings."""
    if sys.maxunicode == 65535:
        return 'UTF16'
    else:
        return 'UTF32'


def analyze_entities(text, encoding='UTF32'):
    body = {
        'document': {
            'type': 'PLAIN_TEXT',
            'content': text,
        },
        'encoding_type': encoding,
    }

    service = get_service()

    request = service.documents().analyzeEntities(body=body)
    response = request.execute()
    entities = []
    for i in range(len(response['entities'])):
    	if(response['entities'][i]['salience']>0.001):
    		entities.append(response['entities'][i]['name'])
    return(set(entities))

In [58]:
interesting_user_entities = []
for i in range(len(interesting_users)):
    interesting_user_entities.append(analyze_entities(interesting_users[i]))

In [82]:
interesting_user_entities[2]

{'Addict',
 'Apple',
 'Estudiante de Espa',
 'Google',
 'Microsoft',
 'Soy',
 'autos',
 'car',
 'cars',
 'fan',
 'honarario',
 'nut',
 'photography',
 'reading fiction',
 'tech',
 'technology'}

In [66]:
for i in range(len(test_output.summary)):
    for j in range(len(test_output["document"][i])):
        if "" in test_output["document"][i][j].split(" "):
            print(i, j)

30 0
30 2
30 3
30 7
30 8
30 9
30 12
30 23
30 24
30 26
30 28
30 29
30 30
30 31
30 32
30 33
92 0
92 2
92 3
92 4
92 5
92 8
92 11
110 3
110 23
110 30
147 0
147 2
147 3
147 6
147 15
147 16
147 20
147 23
147 47
212 9
212 19
212 36
325 1
325 14
325 18
429 8
429 11
491 7
896 0
896 4
896 14
896 31
896 33
896 34
948 31
1120 47
1270 11
1338 0
1338 3
1338 4
1338 7
1338 8
1338 10
1338 11
1338 12
1338 15
1338 16
1338 17
1338 19
1338 20
1338 22
1338 23
1338 24
1338 25
1424 1
1424 4
1424 7
1424 8
1424 13
1424 26
1424 33
1508 6
1520 0
1520 2
1520 3
1520 4
1520 5
1520 9
1520 10
1520 12
1520 13
1520 14
1520 15
1520 17
1520 20
1520 21
1520 22
1520 23
1548 44
1548 45
1564 14
1643 10
1851 1
1851 2
1851 3
1851 4
1851 8
1851 9
1851 11
1851 15
1851 22
1851 53
1851 55
1851 58
1916 5
1988 32
2024 11
2024 18
2041 0
2041 2
2041 3
2041 6
2041 7
2041 12
2041 16
2041 18
2041 19
2041 20
2292 10
2292 13
2292 14
2292 18
2292 19
2292 20
2452 16
2523 9
2541 5
2795 9
2814 19
2814 21
2814 22
2814 23
2884 0
2884 3
2884 4
288

In [62]:
test_output.document[0]

['Neil Redfearn is considering his position as manager of Leeds after the club suspended his assistant Steve Thompson on thursday',
 'Leeds took two months to appoint Steve Thompson but he was told in a letter from director of football Salerno yesterday morning that he was be suspended until the end of the season effectively ending his time at Elland Road after 105 days',
 'Steve Thompson is understood to have been told that he will be suspended until his contract expires at the end of the season',
 'he has also been informed that an option to renew his deal will not be taken up by the club',
 "Leeds assistant manager Steve Thompson ( left ) has been suspended by the club Steve Thompson 's suspension has left manager Neil Redfearn considering his future at Elland Road a club statement read : ' the director of football Salerno , has today ( april 2 ) suspended the assistant coach Steve Thompson from his duties at the club",
 'this is an internal matter and the club will make no further 